In [1]:
import os
import json
import zipfile
import pandas as pd
import json

In [31]:
!pip install -U -q PyDrive


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [37]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials


In [ ]:
#preparing news 

news_folder = os.listdir("/Users/prayagbhatia/Desktop/ASU_Courses/SWM/project/News")
news_folder

In [ ]:
cd /Users/prayagbhatia/Desktop/ASU_Courses/SWM/project/News

In [4]:
cwd = os.getcwd()
cwd

'/Users/prayagbhatia/Desktop/ASU_Courses/SWM/Project/News'

In [ ]:
count = 0
news_list = []
for zf in news_folder:
    print(zf)
    if (zf == '.DS_Store'):
        continue
    with zipfile.ZipFile(zf, 'r') as zip1:
        for file in zip1.namelist():
            df = pd.DataFrame()
            with zip1.open(file) as f:
                count +=1
                data = f.read()
                data = data.decode('utf-8')
                data = json.loads(data)
                if 'apple' in data['title'].lower() or 'amazon' in data['title'].lower() or 'apple' in data['text'].lower() or 'amazon' in data['text'].lower() or 'appl' in data['title'].lower() or 'amzn' in data['title'].lower() or 'appl' in data['text'].lower() or 'amzn' in data['text'].lower() :
                    new_data = {"title" : data["title"],
                               "text" : data["text"],
                               "published_date" : data["published"]}
                
                    news_list.append(data)
                

print(count)
news_df = pd.DataFrame(news_list, columns=['title','text','published'])
news_df

In [11]:
def timezone(value):
  return pd.Timestamp.tz_convert(value,tz='UTC')

In [12]:
news_df["published"]=news_df['published'].apply(pd.to_datetime).apply(timezone)

In [ ]:
news_df.iloc[0:4]

In [ ]:
news_df=news_df.dropna()
news_df.count

In [ ]:
amazon_news_df=news_df[news_df["text"].str.contains("Amazon|Amazon's|AMZN")]
amazon_news_df


In [ ]:
amazon_news_df['Date'] = pd.to_datetime(amazon_news_df['published']).dt.date
amazon_news_df['Time'] = pd.to_datetime(amazon_news_df['published']).dt.time
amazon_news_df

In [ ]:
apple_news_df=news_df[news_df["text"].str.contains("Apple|Appl")]
apple_news_df

In [ ]:
apple_news_df['Date'] = pd.to_datetime(apple_news_df['published']).dt.date
apple_news_df['Time'] = pd.to_datetime(apple_news_df['published']).dt.time
apple_news_df

In [ ]:
applechart=pd.read_csv("/content/drive/MyDrive/Semantic Web Mining/CHARTS/CHARTS/APPLE60.csv",header=None)

In [26]:
def formatted_date(value):
    date_formatted = value.replace(".", "-")
    return date_formatted

In [ ]:
##getting charts

apple_charts = pd.read_csv("/Users/prayagbhatia/Desktop/ASU_Courses/SWM/project/CHARTS/APPLE1440.csv",header=None)
chart_column_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
apple_charts.columns = chart_column_names
apple_charts['DateFormatted'] = apple_charts['Date'].apply(formatted_date).apply(pd.to_datetime).dt.date
apple_charts

In [ ]:
amazon_charts = pd.read_csv("/Users/prayagbhatia/Desktop/ASU_Courses/SWM/project/CHARTS/AMAZON1440.csv",header=None)
chart_column_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
amazon_charts.columns = chart_column_names
amazon_charts['DateFormatted'] = amazon_charts['Date'].apply(formatted_date).apply(pd.to_datetime).dt.date
amazon_charts

amazon_charts.iloc[1084]

In [ ]:
### Mapping Amazon news to amazon charts taking into consideration the Open values
labels = []
for index , row in amazon_news_df.iterrows():
    news_date = row.Date
    match_date_condition = amazon_charts['DateFormatted'] == news_date
    match_idx = amazon_charts.loc[match_date_condition].index
    if len(match_idx):
        match_idx = match_idx[0]
        if amazon_charts.iloc[match_idx-1].Open < amazon_charts.iloc[match_idx].Open:
            labels.append(1)
        else:
            labels.append(0)
    else:
        labels.append(None)
        

amazon_news_df['labels'] = labels
amazon_news_df


In [ ]:
### Mapping Apple news to apple charts taking into consideration the Open values
labels = []
for index , row in apple_news_df.iterrows():
    news_date = row.Date
    match_date_condition = apple_charts['DateFormatted'] == news_date
    match_idx = apple_charts.loc[match_date_condition].index
    if len(match_idx):
        match_idx = match_idx[0]
        if apple_charts.iloc[match_idx-1].Open < apple_charts.iloc[match_idx].Open:
            labels.append(1)
        else:
            labels.append(0)
    else:
        labels.append(None)
        

apple_news_df['labels'] = labels
apple_news_df

In [38]:
scope = ['https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/prayagbhatia/Downloads/creds.json', scope)
gauth = GoogleAuth()
gauth.credentials = credentials

# Create the PyDrive client
drive = GoogleDrive(gauth)



In [ ]:
csv_string = apple_news_df.to_csv()


file = drive.CreateFile({'title': 'applenews3.csv'})
file.SetContentString(csv_string)
file.Upload()


In [22]:
apple_news_df2 = apple_news_df.drop('text', axis=1)
apple_news_df2.to_csv('apple_news2.csv')

In [23]:
amazon_news_df2 = amazon_news_df.drop('text', axis=1)
amazon_news_df2.to_csv('amazon_news2.csv')